# Testing whether missing file or not

In [2]:
import os 
# All files ending with .txt
mypath = "./craw_data/2020-02-01/"

onlyfiles_set = set([f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))])
print(f'total file: {len(onlyfiles_set)}')

base_dir = './craw_data'
list_date = os.listdir(base_dir)
for date in list_date:
	date_folder = os.path.join(base_dir, date)
	date_file_set = set([f for f in os.listdir(date_folder) if (os.path.isfile(os.path.join(date_folder, f)))])
	missing = onlyfiles_set - date_file_set
	if missing:
		print(f'date: {date}, missing: {missing}')


# Integrating and combining data

In [3]:
import pandas as pd 
import numpy as np
import os 
import datetime
from pathlib import Path



In [47]:
def validate(date_text):
	convert_date = ""
	try:
		date = datetime.datetime.strptime(date_text, '%d %b %Y')
		convert_date = datetime.datetime.strftime(date, '%Y-%m-%d')
	except:
		convert_date = None
	return convert_date

# tmp = validate('1 Feb 2021')
# print(tmp)
# print(validate('Time'))

In [48]:
def convert_csv_to_numpy(file_name):
	parent_folder = Path(file_name)
	date = os.path.basename(parent_folder.parent.absolute())
	date = date.split('-')[0]

	siteId = os.path.basename(file_name)
	siteId = siteId.split('.')[0]

	file_pd = pd.read_csv(file_name)
	file_pd = file_pd[file_pd['Time'] == '00-24']
	columns_name = file_pd.columns 
	new_column_names = {}
	for name in columns_name:
		tmp = f'{name} {date}'
		if validate(tmp):
			new_column_names[name] = validate(tmp)

	# select only date column 
	file_pd = file_pd[list(new_column_names.keys())]
	file_pd.rename(columns = new_column_names, inplace=True)

	# convert to format: date, traffict_count
	file_pd = file_pd.transpose()
	file_pd.columns = ['traffic_count']
	file_pd['site_id'] = siteId

	return file_pd.reset_index().to_numpy()


# file_name = os.path.join(date_folder, "1011.csv")
# print(file_name)
# tmp_pd = convert_csv_to_numpy(file_name)
	

In [49]:
save_folder = "./combine_data"
def combine_date_folder_into_one(date_folder):
	list_files = os.listdir(date_folder)
	combine_numpy = np.empty((0, 3))
	for file_name in list_files:
			combine_numpy = np.concatenate((combine_numpy, convert_csv_to_numpy(os.path.join(date_folder, file_name))))
	
	date = os.path.basename(date_folder)
	combine_df = pd.DataFrame(combine_numpy,columns=['Time', 'traffic_count', 'siteId'])

	if not os.path.exists(save_folder):
		os.makedirs(save_folder)

	combine_df.to_csv(os.path.join(save_folder, date), index=False)
	print(f'done date: {date}')


# combine_date_folder_into_one(date_folder)

In [50]:
craw_folder = "./craw_data"
def combine_craw_data_folder():
	list_date_folder = os.listdir(craw_folder)
	list_date_folder.sort()
	for date_folder in list_date_folder:
		combine_date_folder_into_one(os.path.join(craw_folder, date_folder))

# combine_craw_data_folder()

In [51]:
def combine_to_single_file():
	li = []
	all_files = os.listdir(save_folder)
	for filename in all_files:
		filename = os.path.join(save_folder, filename)
		df = pd.read_csv(filename, index_col=None, header=0)
		li.append(df)

	frame = pd.concat(li, axis=0, ignore_index=True)
	frame.to_csv(os.path.join(save_folder, "traffic_count_all.csv"), index=False)

# combine_to_single_file()

# Group by week with avg traffic count

In [53]:
# group data by week 
file_name = '/home/minhnhat/data/combine_data/traffic_count_all.csv'
def group_by_week(file_name):
	df = pd.read_csv(file_name)
	# some traffic_count is -, it means can't craw or some errors happen
	df['traffic_count'] = pd.to_numeric(df['traffic_count'], errors ='coerce').fillna(0).astype('int') 
	df['Time'] = pd.to_datetime(df['Time'])
	group_by_week_df = df.groupby(['siteId', pd.Grouper(key='Time', freq='W-MON')])['traffic_count'].mean().reset_index()
	group_by_week_df.to_csv(os.path.join(save_folder, "traffic_count_group_by_week.csv"), index=False)

	
group_by_week(file_name)

In [54]:
group_by_week_file_name = './combine_data/traffic_count_group_by_week.csv'
test = pd.read_csv(group_by_week_file_name)


In [55]:
test.isnull().sum()

siteId           0
Time             0
traffic_count    0
dtype: int64

In [57]:
test.head()

,siteId,Time,traffic_count
0,1011,2020-02-03,53425.000000
1,1011,2020-02-10,54979.428571
2,1011,2020-02-17,56062.857143
3,1011,2020-02-24,56570.428571
4,1011,2020-03-02,57157.000000


In [42]:
file_name = '/home/minhnhat/data/combine_data/traffic_count_all.csv'

tmp = pd.read_csv(file_name)
tmp.shape

(219488, 3)